In [339]:
from fuzzywuzzy import fuzz
import pandas as pd 
import numpy as np

In [2]:
ent_df = pd.read_csv('entity.csv')
ent_df.head()

,DB_ID,Name,Blck_List
0,14173,ALVAREZ CARLOS,0
1,14174,TARANTOLA SALVATORE,0
2,14175,HOME DEVCO TIVOLI ISLES LLC,1
3,14176,LEVIN MICHAEL,0
4,14177,ROSS NICHOLAS J,0


This code was used to find most commont word tokens.  
I used this to build an ignore list where I am filtering out common  
words to describe a business. IE  Corp, Inc, Co

```python
from collections import Counter
ent_counter = Counter()
for names in ent_df['Name'].str.split():
    ent_counter.update(names)
with open('common.csv', mode='w') as f:
    print("Name, count", file=f)
    for c in  ent_counter.most_common(100):
        print(f'{c[0]},{c[1]}', file=f)
```

In [3]:
ignore = pd.read_csv('common.csv')
ignore_list = ignore['Name'].to_list()

In [4]:
def clean(e):
    return all([t not in ignore_list for t in e.split()])

ent_df_clean = ent_df[ent_df['Name'].map(clean)].copy()
# creating a new pf object not ref

In [5]:
print(f'counts: original -> {len(ent_df)} cleaned -> {len(ent_df_clean)}')

counts: original -> 17187 cleaned -> 13865


In [6]:
ent_df_clean['token_count'] = ent_df_clean['Name'].str.split().str.len()
ent_df_clean.head(10)

,DB_ID,Name,Blck_List,token_count
0,14173,ALVAREZ CARLOS,0,2
1,14174,TARANTOLA SALVATORE,0,2
3,14176,LEVIN MICHAEL,0,2
4,14177,ROSS NICHOLAS J,0,3
5,14178,PISCATELLI ALAN,0,2
6,14179,SCHNEIDER ROBERT,0,2
7,14180,FOBBS RAMONA A,0,3
8,14181,"KELLER FRED ESTATE,SHEPHERD DANIEL J PERSONAL ...",1,7
9,14182,"SHEPHERD DANIEL J PERSONAL REPRESENTATIVE,KELL...",1,7
10,14183,MICHEL JEANTRAL,0,2


In [7]:
groups = ent_df_clean.groupby('token_count').groups

In [9]:
list(groups.items())[0]

(1, Int64Index([ 1418,  2182,  2927,  3868,  5105,  9648,  9732,  9968, 10038,
             10275, 10454, 10976, 11365, 11419, 11690, 11738, 11754, 11780,
             11793, 11959, 12123, 13084, 13921, 14297, 14299, 14314, 14341,
             14477, 14690, 14730, 15020, 15294, 17117],
            dtype='int64'))

In [10]:
ent_df_clean.loc[2182]

DB_ID               16355
Name           BANKUNITED
Blck_List               1
token_count             1
Name: 2182, dtype: object

In [254]:

def test_methods(s):    
    
    results = []
    for s1, s2 in s:
        simple = fuzz.ratio(s1, s2)
        partial = fuzz.partial_ratio(s1, s2)
        token_sort = fuzz.token_sort_ratio(s1, s2)
        token_set = fuzz.token_set_ratio(s1, s2)
        results.append({'simple': simple, 'partial': partial, 'token_sort': token_sort, 'token_set': token_set})
    
    df = pd.DataFrame(results, columns=['simple', 'partial', 'token_sort', 'token_set'])
#     df['simple_weighted'] = df['simple'] * 1
#     df['partial_weighted'] = df['partial'] * (3/4)
#     df['token_sort_weighted'] = df['token_sort'] * .5
#     df['token_set_weighted'] = df['token_set'] * .25
    df = df.append(df.sum().rank(ascending=False), ignore_index=True)
    #return df
    #print(df)
    
    #print(df)
    ranks = pd.Series(df.tail(1).T.iloc[:,-1])
    return ranks.sort_values()

In [255]:
#same tokens simple mispelling
test_methods([('BARNES JAMES', 'BARNS JAMES')])
#simple

simple        2.0
token_sort    2.0
token_set     2.0
partial       4.0
Name: 1, dtype: float64

In [258]:
#same tokens different spelling (nicknames/shorthand)
test_methods([('Albert Zuniga', 'Alberto Zuniga'), ('Mary Smith', 'Marias Smith')])
#use token_sort

simple        2.0
token_sort    2.0
token_set     2.0
partial       4.0
Name: 2, dtype: float64

In [259]:
#tokens different position
test_methods([('Albert Zuniga', 'Zuniga Albert')])
# token sort

token_sort    1.5
token_set     1.5
simple        3.5
partial       3.5
Name: 1, dtype: float64

In [260]:
#different tokens
test_methods([('Albert Zuniga', 'Albert L Zuniga'),('Alberto Zuniga', 'Alberto L Zuniga JR')])

token_set     1.0
simple        2.5
token_sort    2.5
partial       4.0
Name: 2, dtype: float64

In [261]:
#different tokens mispelled
test_methods([('Albert Zunigga', 'Albert L Zuniga'), ('Alberto L Zunniga', 'Alberto L Zuniga JR')])

partial       1.0
simple        3.0
token_sort    3.0
token_set     3.0
Name: 2, dtype: float64

In [295]:
# find same tokens simple mispelling
def join_first(tokens):
    return ''.join([s[0] for s in tokens])
ent_df_clean['grp_1'] = ent_df_clean.Name.str.split().apply(join_first)
#ent_df_clean = ent_df_clean.drop('grp_2', axis=1)

In [337]:
grps_simple = ent_df_clean.groupby('grp_1')
grp_1 = grps_simple.filter(lambda x: x['grp_1'].count() > 1)['grp_1'].unique()
# for grp in grps_simple:
#     if grp[1].size > 1:
#         print(grp[0])

In [338]:
grp_1

array(['AC', 'TS', 'LM', ..., 'MWP', 'CJF', 'JMT'], dtype=object)

In [341]:
ent_df_clean['alias_match'], ent_df_clean['score'] = np.nan, np.nan

In [ ]:
 for group_name in grp_1:
    this_group = grps_simple.get_group('AC').sort_values('Name')
    for lidx in this_group.index:
        for ridx in this_group.index:
            if lidx == ridx: 
                continue
            if pd.isna(this_group['alias_match'].loc[lidx]):
                this_group['alias_match'].loc[lidx] = this_group['Name'].loc[lidx]
                this_group['score'].loc[lidx] = 110
            if pd.isna(this_group['alias_match'].loc[ridx]):
                score = fuzz.ratio(this_group['Name'].loc[lidx], this_group['Name'].loc[ridx])
                if score > 85:
                    this_group['alias_match'].loc[ridx] = this_group['Name'].loc[lidx]
                    this_group['score'].loc[ridx] = score



C:\Users\Al\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\Al\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\Al\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\Al\Anac